# EV vs Actual PnL Gap

Inspect existing run artifacts to understand how EV estimates compare to realized PnL. Use this notebook to explore available metrics and highlight any missing logging.

In [2]:
from pathlib import Path
import json
import pandas as pd
from IPython.display import display

PROJECT_ROOT = Path.cwd().resolve()
search_roots = [PROJECT_ROOT] + list(PROJECT_ROOT.parents)
for root in search_roots:
    if (root / 'runs').exists():
        PROJECT_ROOT = root
        break
else:
    checked = ', '.join(str(p) for p in search_roots)
    raise FileNotFoundError('Could not locate runs directory. Checked: ' + checked)

RUNS_DIR = PROJECT_ROOT / 'runs'
DAILY_PATH = PROJECT_ROOT / 'daily.csv'

if not DAILY_PATH.exists():
    raise FileNotFoundError(f'Expected daily.csv at {DAILY_PATH}; verify data export.')



In [3]:
def load_records(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    if 'ts' in df.columns:
        df['ts'] = pd.to_datetime(df['ts'], errors='coerce')
    numeric_cols = ['ev_lcb', 'pnl_pips', 'cost_pips', 'slip_est', 'slip_real', 'tp_pips', 'sl_pips', 'or_atr_ratio', 'min_or_atr_ratio']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def load_state(path: Path) -> dict:
    with path.open() as fh:
        return json.load(fh)


In [4]:
record_paths = sorted(RUNS_DIR.glob('*/records.csv'))
print(f'Found {len(record_paths)} run(s) with records.csv')
if record_paths:
    display(pd.DataFrame({'run_id': [p.parent.name for p in record_paths], 'path': record_paths}))
else:
    print('No records.csv files detected. Add logging or rerun export before proceeding.')


Found 10 run(s) with records.csv


,run_id,path
0,USDJPY_conservative_20250922_143631,/Users/izumimotohayato/development/invest3_orb...
1,USDJPY_conservative_20250922_143714,/Users/izumimotohayato/development/invest3_orb...
2,USDJPY_conservative_20250922_143756,/Users/izumimotohayato/development/invest3_orb...
3,USDJPY_conservative_20250922_150418,/Users/izumimotohayato/development/invest3_orb...
4,USDJPY_conservative_20250922_152942,/Users/izumimotohayato/development/invest3_orb...
5,USDJPY_conservative_20250922_153303,/Users/izumimotohayato/development/invest3_orb...
6,USDJPY_conservative_20250922_170854,/Users/izumimotohayato/development/invest3_orb...
7,USDJPY_conservative_20250922_170914,/Users/izumimotohayato/development/invest3_orb...
8,USDJPY_conservative_20250922_175708,/Users/izumimotohayato/development/invest3_orb...
9,USDJPY_conservative_20250923_202605,/Users/izumimotohayato/development/invest3_orb...


### 対象ランの選択
`TARGET_RUN_ID` を指定すると、そのランの `records.csv`/`state.json` を読み込みます。未指定の場合は最新のランを使用します。


In [ ]:
TARGET_RUN_ID = None  # 例: 'USDJPY_conservative_20250923_202605'
if record_paths:
    default_run = record_paths[-1].parent.name
    if TARGET_RUN_ID is None:
        TARGET_RUN_ID = default_run
    elif TARGET_RUN_ID not in [p.parent.name for p in record_paths]:
        raise ValueError(f'TARGET_RUN_ID={TARGET_RUN_ID} が runs ディレクトリに見つかりません。候補: {[p.parent.name for p in record_paths]}')
    print(f'Using TARGET_RUN_ID: {TARGET_RUN_ID}')
else:
    TARGET_RUN_ID = None
    print('record_paths is empty. TARGET_RUN_ID は設定できません。')


In [5]:
if TARGET_RUN_ID:
    sample_candidate = RUNS_DIR / TARGET_RUN_ID / 'records.csv'
    if not sample_candidate.exists():
        raise FileNotFoundError(f'records.csv not found for TARGET_RUN_ID={TARGET_RUN_ID}')
    sample_path = sample_candidate
elif record_paths:
    sample_path = record_paths[-1]
else:
    sample_path = None

if sample_path is None:
    raise SystemExit('No run artifacts to inspect yet.')

TARGET_RUN_ID = sample_path.parent.name
print(f'Sampling run: {TARGET_RUN_ID}')
records_df = load_records(sample_path)
print(f'Record count: {len(records_df)}')
display(records_df.head())
try:
    display(records_df.describe(include='all', datetime_is_numeric=True))
except TypeError:
    display(records_df.describe(include='all'))



Sampling run: USDJPY_conservative_20250923_202605
Record count: 5000


,cost_base,cost_pips,entry_ts,ev_lcb,ev_pass,exit,expected_slip_pip,min_or_atr_ratio,or_atr_ratio,pnl_pips,...,session,side,sl_pips,slip_est,slip_real,spread_band,stage,threshold_lcb,tp_pips,ts
0,0.5,0.5,2018-01-02 08:25:00,-0.436142,False,sl,0.0,0.6,4.501182,-2.917143,...,LDN,SELL,2.417143,0.0,0.0,narrow,trade,0.522630,3.021429,2018-01-02 08:25:00
1,0.5,0.5,2018-01-02 08:30:00,-0.491247,False,tp,0.0,0.6,3.933884,1.457143,...,LDN,SELL,2.765714,0.0,1.5,narrow,trade,0.492244,3.457143,2018-01-02 08:30:00
2,0.5,0.5,2018-01-02 08:35:00,-0.436679,False,tp,0.0,0.6,3.800399,1.578571,...,LDN,SELL,2.862857,0.0,1.5,narrow,trade,0.483776,3.578571,2018-01-02 08:35:00
3,0.5,0.5,2018-01-02 08:40:00,-0.383096,False,tp,0.0,0.6,3.792829,1.585714,...,LDN,SELL,2.868571,0.0,1.5,narrow,trade,0.483278,3.585714,2018-01-02 08:40:00
4,0.5,0.5,2018-01-02 08:45:00,-0.329864,False,tp,0.0,0.6,3.777778,1.600000,...,LDN,SELL,2.880000,0.0,1.5,narrow,trade,0.482282,3.600000,2018-01-02 08:45:00


,cost_base,cost_pips,entry_ts,ev_lcb,ev_pass,exit,expected_slip_pip,min_or_atr_ratio,or_atr_ratio,pnl_pips,...,session,side,sl_pips,slip_est,slip_real,spread_band,stage,threshold_lcb,tp_pips,ts
count,138.0,1635.0,138,1571.000000,138,202,138.0,1.780000e+02,1767.000000,202.000000,...,138,3390,1631.000000,202.0,202.000000,1727,5000,138.000000,1631.000000,5000
unique,NaN,NaN,138,NaN,2,2,NaN,NaN,NaN,NaN,...,2,2,NaN,NaN,NaN,1,7,NaN,NaN,NaN
top,NaN,NaN,2018-01-02 08:25:00,NaN,True,tp,NaN,NaN,NaN,NaN,...,LDN,BUY,NaN,NaN,NaN,narrow,no_breakout,NaN,NaN,NaN
freq,NaN,NaN,1,NaN,107,148,NaN,NaN,NaN,NaN,...,136,1728,NaN,NaN,NaN,1727,1610,NaN,NaN,NaN
mean,0.5,0.5,NaN,-0.053023,NaN,NaN,0.0,6.000000e-01,2.815214,-0.113458,...,NaN,NaN,3.357187,0.0,1.226238,NaN,NaN,0.326204,4.196483,2018-01-22 19:14:49.860000
min,0.5,0.5,NaN,-0.491247,NaN,NaN,0.0,6.000000e-01,0.000000,-46.474286,...,NaN,NaN,0.908571,0.0,0.000000,NaN,NaN,0.000000,1.135714,2018-01-01 22:00:00
25%,0.5,0.5,NaN,-0.250421,NaN,NaN,0.0,6.000000e-01,1.550507,-3.745714,...,NaN,NaN,2.194286,0.0,1.500000,NaN,NaN,0.189271,2.742857,2018-01-08 01:48:45
50%,0.5,0.5,NaN,-0.126098,NaN,NaN,0.0,6.000000e-01,2.388889,1.064286,...,NaN,NaN,2.851429,0.0,1.500000,NaN,NaN,0.357254,3.564286,2018-01-12 10:37:30
75%,0.5,0.5,NaN,0.057542,NaN,NaN,0.0,6.000000e-01,3.226277,1.585714,...,NaN,NaN,4.080000,0.0,1.500000,NaN,NaN,0.502456,5.100000,2018-01-18 19:06:15
max,0.5,0.5,NaN,2.706772,NaN,NaN,0.0,6.000000e-01,13.496403,25.692857,...,NaN,NaN,44.474286,0.0,1.500000,NaN,NaN,0.583400,55.592857,2022-12-20 08:25:00


In [6]:
stage_counts = records_df['stage'].value_counts(dropna=False) if 'stage' in records_df else pd.Series(dtype=int)
print('Stage distribution:')
display(stage_counts)

if 'pnl_pips' in records_df:
    trade_mask = records_df['pnl_pips'].notna()
else:
    trade_mask = pd.Series(False, index=records_df.index)
trade_records = records_df.loc[trade_mask].copy()
print(f'Realized trade rows: {len(trade_records)}')
if len(trade_records) == 0:
    print('No realized PnL entries yet — consider enhancing logging if this persists.')
else:
    if 'ev_lcb' in trade_records:
        trade_records['ev_gap'] = trade_records['pnl_pips'] - trade_records['ev_lcb']
    else:
        trade_records['ev_gap'] = pd.NA
    display(trade_records[['ts', 'stage', 'ev_lcb', 'pnl_pips', 'ev_gap']].head())



Stage distribution:


stage
no_breakout    1610
gate_block     1589
ev_reject      1433
trade           198
rv_filter       126
or_filter        40
trade_exit        4
Name: count, dtype: int64

Realized trade rows: 202


,ts,stage,ev_lcb,pnl_pips,ev_gap
0,2018-01-02 08:25:00,trade,-0.436142,-2.917143,-2.481001
1,2018-01-02 08:30:00,trade,-0.491247,1.457143,1.948390
2,2018-01-02 08:35:00,trade,-0.436679,1.578571,2.015251
3,2018-01-02 08:40:00,trade,-0.383096,1.585714,1.968811
4,2018-01-02 08:45:00,trade,-0.329864,1.600000,1.929864


In [7]:
state_path = sample_path.with_name('state.json')
if not state_path.exists():
    print('State file missing; EV profile unavailable for this run.')
else:
    state = load_state(state_path)
    ev_global = state.get('ev_global', {})
    ev_buckets = state.get('ev_buckets', {})
    print('Global EV parameters')
    display(pd.Series(ev_global))
    if ev_buckets:
        ev_bucket_df = pd.DataFrame.from_dict(ev_buckets, orient='index')
        display(ev_bucket_df)
    else:
        print('No bucket-level EV parameters recorded')


Global EV parameters


alpha          27.677216
beta           22.322784
prior_alpha     0.000000
prior_beta      0.000000
decay           0.020000
conf            0.950000
dtype: float64

,alpha,beta
LDN:narrow:high,28.115997,21.884000
LDN:narrow:low,1.000000,1.000000
LDN:narrow:mid,10.274647,9.564684
NY:narrow:high,11.289983,11.992418
NY:narrow:low,1.000000,1.000000
NY:narrow:mid,27.486488,22.513499
TOK:narrow:high,1.000000,1.000000
TOK:narrow:low,1.000000,1.000000
TOK:narrow:mid,1.000000,1.000000


In [8]:
daily_df = pd.read_csv(DAILY_PATH, parse_dates=['date'])
daily_df = daily_df.sort_values('date')
print(f'Daily rows: {len(daily_df)}')
display(daily_df.head())
display(daily_df.tail())


Daily rows: 2190


,date,breakouts,gate_pass,gate_block,ev_pass,ev_reject,fills,wins,pnl_pips
0,2018-01-01,9,0,9,0,0,0,0,0.0
1,2018-01-02,253,131,122,0,131,0,0,0.0
2,2018-01-03,172,128,44,0,128,0,0,0.0
3,2018-01-04,215,102,113,0,102,0,0,0.0
4,2018-01-05,245,106,139,0,106,0,0,0.0


,date,breakouts,gate_pass,gate_block,ev_pass,ev_reject,fills,wins,pnl_pips
2185,2024-12-26,265,146,119,0,146,0,0,0.0
2186,2024-12-27,204,106,98,0,106,0,0,0.0
2187,2024-12-29,12,0,12,0,0,0,0,0.0
2188,2024-12-30,248,118,130,0,118,0,0,0.0
2189,2024-12-31,242,146,96,0,146,0,0,0.0


In [9]:
if 'trade_records' not in globals():
    print('trade_records is undefined. Run the preparation cells above first.')
elif trade_records.empty:
    print('Daily aggregation skipped because no realized trade records were found.')
else:
    trade_daily = (trade_records
        .dropna(subset=['ts'])
        .assign(date=lambda d: d['ts'].dt.floor('D'))
        .groupby('date')[['pnl_pips', 'ev_lcb']]
        .sum()
        .rename(columns={'pnl_pips': 'realized_pnl_pips', 'ev_lcb': 'ev_lcb_sum'})
        .reset_index())
    merged = daily_df.merge(trade_daily, how='left', on='date')
    display(merged.head())
    print('Columns available for plotting EV vs realized PnL:')
    display(merged.columns)



,date,breakouts,gate_pass,gate_block,ev_pass,ev_reject,fills,wins,pnl_pips,realized_pnl_pips,ev_lcb_sum
0,2018-01-01,9,0,9,0,0,0,0,0.0,NaN,NaN
1,2018-01-02,253,131,122,0,131,0,0,0.0,91.080000,15.161257
2,2018-01-03,172,128,44,0,128,0,0,0.0,NaN,NaN
3,2018-01-04,215,102,113,0,102,0,0,0.0,2.128571,1.072088
4,2018-01-05,245,106,139,0,106,0,0,0.0,NaN,NaN


Columns available for plotting EV vs realized PnL:


Index(['date', 'breakouts', 'gate_pass', 'gate_block', 'ev_pass', 'ev_reject',
       'fills', 'wins', 'pnl_pips', 'realized_pnl_pips', 'ev_lcb_sum'],
      dtype='object')

In [ ]:
if 'merged' in globals():
    eval_df = merged.dropna(subset=['realized_pnl_pips', 'ev_lcb_sum']).copy()
    if eval_df.empty:
        print('No overlapping daily data to summarize for the selected run.')
    else:
        eval_df['ev_gap'] = eval_df['realized_pnl_pips'] - eval_df['ev_lcb_sum']
        summary = {
            'days_with_records': int(len(eval_df)),
            'total_realized_pips': float(eval_df['realized_pnl_pips'].sum()),
            'total_ev_pips': float(eval_df['ev_lcb_sum'].sum()),
            'total_gap_pips': float(eval_df['ev_gap'].sum()),
            'mean_daily_gap': float(eval_df['ev_gap'].mean()),
            'max_positive_gap': float(eval_df['ev_gap'].max()),
            'max_negative_gap': float(eval_df['ev_gap'].min()),
        }
        print('Summary (selected run)')
        display(pd.Series(summary))
        top_n = min(5, len(eval_df))
        if top_n:
            print('Top positive gap days:')
            display(eval_df.nlargest(top_n, 'ev_gap')[['date', 'realized_pnl_pips', 'ev_lcb_sum', 'ev_gap']])
            print('Top negative gap days:')
            display(eval_df.nsmallest(top_n, 'ev_gap')[['date', 'realized_pnl_pips', 'ev_lcb_sum', 'ev_gap']])
else:
    print('Run the aggregation cell to create `merged` first.')



In [ ]:
def combine_runs_record(record_paths):
    frames = []
    for path in record_paths:
        df = load_records(path)
        df['run_id'] = path.parent.name
        frames.append(df)
    if not frames:
        return pd.DataFrame()
    return pd.concat(frames, ignore_index=True)

all_records_df = combine_runs_record(record_paths)
print(f'Combined records: {len(all_records_df)} rows across {all_records_df["run_id"].nunique()} run(s)')
if not all_records_df.empty:
    combined_trade_records = all_records_df.loc[all_records_df['pnl_pips'].notna()].copy()
    if 'ts' in combined_trade_records:
        combined_trade_records['date'] = combined_trade_records['ts'].dt.floor('D')
    grouped = (combined_trade_records
        .groupby(['run_id', 'date'])[['pnl_pips', 'ev_lcb']]
        .sum()
        .rename(columns={'pnl_pips': 'realized_pnl_pips', 'ev_lcb': 'ev_lcb_sum'})
        .reset_index())
    grouped['ev_gap'] = grouped['realized_pnl_pips'] - grouped['ev_lcb_sum']
    display(grouped.head())

    run_summary = (grouped.groupby('run_id')
        .agg(
            days_with_pnl=('date', 'nunique'),
            realized_total=('realized_pnl_pips', 'sum'),
            ev_total=('ev_lcb_sum', 'sum'),
            gap_total=('ev_gap', 'sum'),
            mean_gap=('ev_gap', 'mean'),
            max_gap=('ev_gap', 'max'),
            min_gap=('ev_gap', 'min'),
        )
        .reset_index())
    print('Per-run summary:')
    display(run_summary)

    overall = {
        'runs': int(run_summary['run_id'].nunique()),
        'days_with_pnl': int(grouped['date'].nunique()),
        'realized_total': float(run_summary['realized_total'].sum()),
        'ev_total': float(run_summary['ev_total'].sum()),
        'gap_total': float(run_summary['gap_total'].sum()),
    }
    print('Overall across runs:')
    display(pd.Series(overall))
else:
    print('No trade records present across runs.')



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
if 'merged' in globals():
    plot_df = merged.dropna(subset=['realized_pnl_pips', 'ev_lcb_sum']).copy()
    if plot_df.empty:
        print('Merged dataframe has no rows with both realized and EV values.')
    else:
        fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
        axes[0].plot(plot_df['date'], plot_df['realized_pnl_pips'], label='Realized PnL', marker='o', linewidth=1)
        axes[0].plot(plot_df['date'], plot_df['ev_lcb_sum'], label='EV Sum', marker='o', linewidth=1)
        axes[0].axhline(0, color='black', linewidth=0.8, linestyle='--')
        axes[0].set_ylabel('Pips')
        axes[0].legend()
        axes[0].set_title('EV vs Realized PnL (Daily)')

        axes[1].bar(plot_df['date'], plot_df['realized_pnl_pips'] - plot_df['ev_lcb_sum'], color='steelblue')
        axes[1].axhline(0, color='black', linewidth=0.8, linestyle='--')
        axes[1].set_ylabel('Gap (Realized - EV)')
        axes[1].set_title('Daily Gap')
        plt.tight_layout()

        plt.figure(figsize=(6, 6))
        sns.scatterplot(x='ev_lcb_sum', y='realized_pnl_pips', data=plot_df)
        plt.axline((0, 0), slope=1, color='black', linestyle='--')
        plt.xlabel('EV Sum (Daily)')
        plt.ylabel('Realized PnL (Daily)')
        plt.title('EV vs Realized PnL Scatter')
        plt.tight_layout()
else:
    print('Run previous cells to create `merged` dataframe first.')



## Next Steps
- `TARGET_RUN_ID` を変更して対象ランの EV vs 実績を切り替えられます。
- `merged` / `eval_df` を活用し、必要に応じて追加のチャートや統計を拡張してください。
- 複数ランを比較したい場合は multi-run 集計セルを実行し、まとめ結果を確認します。
